In [13]:
from geopy.geocoders import Nominatim
import pandas as pd
geolocator = Nominatim(user_agent="My-Python-Script")


df = pd.read_excel("NEW RETAILER INFORMATIONS.xlsx")

In [14]:
def reverse_geocode(latitude, longitude):
    location = geolocator.reverse((latitude, longitude))
    return location.address if location else None


def geocode_address(address):
    # Create a geolocator
    geolocator = Nominatim(user_agent="My-Python-Script")
    try:
        location = geolocator.geocode(address)

        # If the address was found, get the latitude and longitude
        if location:
            latitude = location.latitude
            longitude = location.longitude
            return latitude, longitude
    except:
        # If the address was not found, return None
        return None, None

In [15]:
def handle_missing_street(df):
    
    for idx, row in df.iterrows():
        # Check if the street is missing
        if pd.isna(row["street"]):
            # If the street is missing, try to find it using the lat and long
            latitude, longitude = row["latitude"], row["longitude"]
            address = reverse_geocode(latitude, longitude)
            # If the address is found, update the street with the retrieved address
            if address:
                df.at[idx, "street"] = address

    return df

In [16]:
def handle_missing_coordinates(df):

    # Loop through the rows in the dataframe
    for idx, row in df.iterrows():
        # Check if the latitude and longitude are missing
        if pd.isna(row["latitude"]) and pd.isna(row["longitude"]):
            # If both latitude and longitude are missing, try to find them using the street address
            postal_code = row["postal_code"]
            city = row["city"]
            street = row["street"]
            address = f"{postal_code} {city}"

            try:
                latitude, longitude = geocode_address(address)
            except Exception as e:
                print(f"Error while geocoding address '{address}': {e}")
                latitude = None
                longitude = None

            # If the coordinates were found, update the latitude and longitude with the retrieved coordinates
            if latitude and longitude:
                df.at[idx, "latitude"] = latitude
                df.at[idx, "longitude"] = longitude

    return df

In [17]:
df = handle_missing_coordinates(df)
df = handle_missing_street(df)

Error while geocoding address '57450.0 FARÉBERSVILER': cannot unpack non-iterable NoneType object
Error while geocoding address '77410.0 CLAYES-SOUILLY': cannot unpack non-iterable NoneType object


In [19]:
df.to_excel('df.xlsx',index=False)